In [ ]:
import asyncio
import os
import subprocess
import sys
import pandas as pd
import re
import json
import random

# --- 1. УСТАНОВКА БИБЛИОТЕК ---
def install_libs():
    try:
        import playwright
        import bs4
    except ImportError:
        print("⏳ Устанавливаем библиотеки...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "playwright", "beautifulsoup4", "pandas"])
        os.system("playwright install chromium")
        os.system("playwright install-deps")
        print("✅ Библиотеки готовы.")

install_libs()

from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

# ==========================================
# ЧАСТЬ 1: ДРАМАТИЧЕСКИЙ ТЕАТР (DRAMTEATR)
# ==========================================

async def scrape_dram_schedule(page):
    url = "https://dramteatr39.ru/afisha"
    print(f"\n🎭 [Драмтеатр] Этап 1: Сканируем расписание на {url}...")

    try:
        await page.goto(url, timeout=90000, wait_until='domcontentloaded')
        await page.wait_for_timeout(3000)

        # Прокрутка
        scrolls = 6
        for _ in range(scrolls):
            await page.mouse.wheel(0, 4000)
            await page.wait_for_timeout(random.randint(1000, 3000))

        content = await page.content()
        soup = BeautifulSoup(content, 'html.parser')

        events_list = []
        event_links = soup.find_all('a', href=re.compile(r'/spektakli/'))
        seen_identifiers = set()

        for link in event_links:
            href = link.get('href')
            full_link = f"https://dramteatr39.ru{href}" if href.startswith('/') else href

            title = link.get_text(strip=True)
            if len(title) < 2:
                continue

            container = link.find_parent(class_='affiche__container')
            card = container
            date_block = None
            if container:
                date_block = container.select_one('.affiche__date-block')
            else:
                # Fallback for unexpected markup changes.
                card = link.find_parent()
                if card:
                    date_block = card.select_one('.affiche__date-block')

            if not card:
                continue

            card_text = card.get_text(" ", strip=True).upper()

            # Дата
            date_text = "Дата не определена"
            day_text = ""
            month_text = ""
            time_text = ""

            if date_block:
                day_node = date_block.select_one('.date-list__day')
                month_node = date_block.select_one('.date-list__month')
                time_node = date_block.select_one('.date-list__time')

                if day_node:
                    day_text = day_node.get_text(strip=True)
                if month_node:
                    month_text = month_node.get_text(strip=True)
                if time_node:
                    spans = [s.get_text(strip=True) for s in time_node.find_all('span') if s.get_text(strip=True)]
                    if spans:
                        time_text = spans[-1]
                    else:
                        raw_time = time_node.get_text(" ", strip=True)
                        parts = [p for p in raw_time.split() if ":" in p]
                        if parts:
                            time_text = parts[-1]

            if day_text and month_text:
                date_text = f"{day_text} {month_text}"
                if time_text:
                    date_text = f"{date_text} {time_text}"

            # Статус
            status = "available"
            if "БИЛЕТОВ НЕТ" in card_text or "ПРОДАНО" in card_text:
                status = "sold_out"

            # Сцена
            if "ОСНОВНАЯ" in card_text:
                scene = "Основная"
            elif "СРЕДНЯЯ" in card_text:
                scene = "Средняя"
            else:
                scene = "Малая"

            event_id = f"{date_text}_{title}"
            if event_id in seen_identifiers:
                continue
            seen_identifiers.add(event_id)

            events_list.append({
                "title": title,
                "date_raw": date_text,
                "ticket_status": status,
                "scene": scene,
                "url": full_link,
                "location": "Драматический театр"
            })

        print(f"✅ [Драмтеатр] Найдено событий: {len(events_list)}")
        return events_list
    except Exception as e:
        print(f"❌ [Драмтеатр] Ошибка расписания: {e}")
        return []

async def scrape_dram_details(context, unique_urls):
    print(f"🕵️‍♂️ [Драмтеатр] Сбор деталей для {len(unique_urls)} спектаклей...")
    details_map = {}
    page = await context.new_page()

    for url in unique_urls:
        try:
            await page.goto(url, timeout=45000, wait_until='domcontentloaded')
            try:
                await page.wait_for_selector('h1', timeout=3000)
            except:
                pass

            content = await page.content()
            soup = BeautifulSoup(content, 'html.parser')
            full_text = soup.get_text(" ", strip=True).upper()

            is_pushkin = "ПУШКИНСКАЯ КАРТА" in full_text

            images = []
            for a in soup.find_all('a', href=re.compile(r'\.(jpg|jpeg|png)$', re.IGNORECASE)):
                img_url = a['href']
                if img_url.startswith('/'):
                    img_url = f"https://dramteatr39.ru{img_url}"
                if img_url not in images:
                    images.append(img_url)

            if len(images) < 2:
                for img in soup.select('div.swiper-slide img, .gallery img, .content img'):
                    src = img.get('src')
                    if src and 'logo' not in src:
                        if src.startswith('/'):
                            src = f"https://dramteatr39.ru{src}"
                        if src not in images:
                            images.append(src)

            desc_text = ""
            desc_div = soup.find('div', class_=re.compile(r'description|text-block'))
            if not desc_div:
                header = soup.find(string=re.compile("О спектакле"))
                if header and header.parent:
                    parent = header.find_parent('div')
                    if parent:
                        desc_div = parent
            if desc_div:
                desc_text = desc_div.get_text("\n", strip=True)

            creators = ""
            c_div = soup.find('div', class_=re.compile(r'creators|team'))
            if c_div:
                creators = c_div.get_text(" | ", strip=True)

            details_map[url] = {
                "pushkin_card": is_pushkin,
                "photos": images,
                "description": desc_text[:1500],
                "creators": creators[:500]
            }
        except Exception:
            details_map[url] = {}

    await page.close()
    return details_map

async def run_dramteatr(browser):
    context = await browser.new_context(viewport={'width': 1920, 'height': 1080})
    page = await context.new_page()

    schedule = await scrape_dram_schedule(page)
    if not schedule:
        await context.close()
        return []

    unique_links = list(set(x['url'] for x in schedule))
    details = await scrape_dram_details(context, unique_links)

    await context.close()

    final_data = []
    for item in schedule:
        det = details.get(item['url'], {})
        final_data.append({**item, **det})

    return final_data

# ==========================================
# ЧАСТЬ 2: МУЗЫКАЛЬНЫЙ ТЕАТР (MUZTEATR)
# ==========================================

BASE_URL_MUZ = "https://muzteatr39.ru"

async def find_muz_afisha_link(page):
    try:
        await page.goto(BASE_URL_MUZ, timeout=30000, wait_until='domcontentloaded')
        link = page.get_by_role("link", name=re.compile("Афиша", re.IGNORECASE)).first
        href = await link.get_attribute("href")
        if href:
            return f"{BASE_URL_MUZ}{href}" if href.startswith("/") else href
    except:
        pass
    return f"{BASE_URL_MUZ}/afisha"

async def scrape_muz_schedule(page, afisha_url):
    print(f"\n🎻 [Музтеатр] Этап 1: Сканируем афишу: {afisha_url}...")
    try:
        await page.goto(afisha_url, timeout=60000, wait_until='domcontentloaded')
        await page.wait_for_timeout(2000)
    except:
        pass

    # Скролл
    try:
        for _ in range(3):
            await page.mouse.wheel(0, 4000)
            await page.wait_for_timeout(1000)
    except:
        pass

    content = await page.content()
    soup = BeautifulSoup(content, 'html.parser')

    events_list = []
    unique_ids = set()
    MONTHS_MAP = {'ЯНВАР': 'ЯНВАРЯ', 'ФЕВРАЛ': 'ФЕВРАЛЯ', 'МАРТ': 'МАРТА', 'АПРЕЛ': 'АПРЕЛЯ', 'МАЙ': 'МАЯ', 'ИЮН': 'ИЮНЯ', 'ИЮЛ': 'ИЮЛЯ', 'АВГУСТ': 'АВГУСТА', 'СЕНТЯБР': 'СЕНТЯБРЯ', 'ОКТЯБР': 'ОКТЯБРЯ', 'НОЯБР': 'НОЯБРЯ', 'ДЕКАБР': 'ДЕКАБРЯ'}
    last_seen_month = "?"

    rows = soup.find_all('div', class_=re.compile(r'row\s+afisha'))
    for row in rows:
        date_col = row.find('div', class_=re.compile(r'afisha_data'))
        day_str, month_str, time_str = "?", last_seen_month, "00:00"

        if date_col:
            col_text = date_col.get_text(" ", strip=True).upper()
            day_div = date_col.find('div', class_='afisha_nom')
            if day_div:
                day_str = day_div.get_text(strip=True)
            else:
                day_match = re.search(r'^(\d{1,2})', col_text)
                if day_match:
                    day_str = day_match.group(1)

            for root, full_name in MONTHS_MAP.items():
                if root in col_text:
                    month_str = full_name
                    last_seen_month = full_name
                    break

            time_match = re.search(r'(\d{1,2}:\d{2})', col_text)
            if time_match:
                time_str = time_match.group(1).zfill(5)

        if not day_str.isdigit():
            continue
        date_final = f"{day_str} {month_str} {time_str}"

        title_div = row.find('div', class_='afisha_spek_title')
        if not title_div:
            continue

        title = ""
        link_tag = title_div.find('a')
        if link_tag:
            title = link_tag.get_text(strip=True)
            href = link_tag.get('href')
        else:
            badge = title_div.find('span', class_='badge')
            if badge:
                badge.decompose()
            title = title_div.get_text(strip=True)
            href = None

        if not href:
            continue
        full_link = f"{BASE_URL_MUZ}{href}" if href.startswith('/') else href

        badge = row.find('span', class_='badge')
        age = badge.get_text(strip=True) if badge else ""

        row_text_full = row.get_text(" ", strip=True).upper()
        status = "available"
        if "БИЛЕТОВ НЕТ" in row_text_full or "ПРОДАНО" in row_text_full:
            status = "sold_out"

        eid = f"{date_final}_{full_link}"
        if eid in unique_ids:
            continue
        unique_ids.add(eid)

        events_list.append({
            "title": title,
            "date_raw": date_final,
            "age_restriction": age,
            "ticket_status": status,
            "url": full_link,
            "location": "Музыкальный театр"
        })

    print(f"✅ [Музтеатр] Найдено событий: {len(events_list)}")
    return events_list

async def scrape_muz_details(context, unique_urls):
    print(f"🕵️‍♂️ [Музтеатр] Сбор деталей для {len(unique_urls)} событий...")
    details = {}
    page = await context.new_page()
    BAD_IMAGES = ['ulogin', 'provider', 'icon', 'logo', 'social', 'vk.com', 'ok.ru', 'blank', 'pixel']

    for url in unique_urls:
        try:
            if not url or 'http' not in url:
                continue
            try:
                await page.goto(url, timeout=30000, wait_until='domcontentloaded')
            except:
                pass

            content = await page.content()
            soup = BeautifulSoup(content, 'html.parser')

            desc_text = ""
            meta_og = soup.find("meta", property="og:description")
            if meta_og and meta_og.get("content"):
                desc_text = meta_og["content"].strip()
            if not desc_text:
                meta_desc = soup.find("meta", attrs={"name": "description"})
                if meta_desc and meta_desc.get("content"):
                    desc_text = meta_desc["content"].strip()
            if not desc_text:
                content_div = soup.find('div', class_=re.compile(r'detail-text|news-detail|item-text'))
                if content_div:
                    raw = content_div.get_text("\n", strip=True)
                    lines = [l for l in raw.split('\n') if len(l) > 30 and 'войти' not in l.lower()]
                    desc_text = "\n".join(lines)[:2000]

            images = []
            for d in soup.find_all('div', style=re.compile(r'url\(')):
                match = re.search(r'url\([\'\"]?(.*?)[\'\"]?\)', d['style'])
                if match:
                    u = match.group(1)
                    if u.startswith('/'):
                        u = f"{BASE_URL_MUZ}{u}"
                    if not any(b in u.lower() for b in BAD_IMAGES):
                        images.append(u)

            container = soup.find('div', class_=re.compile(r'container|main|content')) or soup
            for img in container.find_all('img'):
                src = img.get('src')
                if src and not src.endswith('.svg') and not any(b in src.lower() for b in BAD_IMAGES):
                    if src.startswith('/'):
                        src = f"{BASE_URL_MUZ}{src}"
                    images.append(src)

            details[url] = {
                "pushkin_card": "ПУШКИНСКАЯ КАРТА" in soup.get_text().upper(),
                "description": desc_text,
                "photos": list(set(images))
            }
        except:
            details[url] = {}

    await page.close()
    return details

async def run_muzteatr(browser):
    context = await browser.new_context(viewport={'width': 1920, 'height': 1080})
    page = await context.new_page()

    afisha_url = await find_muz_afisha_link(page)
    events = await scrape_muz_schedule(page, afisha_url)

    if not events:
        await context.close()
        return []

    urls = list(set(x['url'] for x in events))
    details = await scrape_muz_details(context, urls)

    await context.close()

    final = []
    for ev in events:
        det = details.get(ev['url'], {})
        final.append({**ev, **det})
    return final

BASE_URL_SOBOR = "https://sobor39.ru"

# --- 2. ПАРСЕР СОБОРА ---
async def scrape_sobor_strict(page):
    target_url = "https://sobor39.ru/events/concerts/night/"
    print(f"⛪ Заходим на {target_url}...")

    try:
        await page.goto(target_url, timeout=90000, wait_until='domcontentloaded')
        await page.wait_for_timeout(5000)
    except Exception as e:
        print(f"⚠️ Предупреждение загрузки: {e}. Пробуем работать с тем, что есть.")

    # --- ПРОКРУТКА ---
    print("⬇️ Начинаем прокрутку для подгрузки событий...")

    for i in range(10):
        await page.mouse.wheel(0, 5000)
        await page.wait_for_timeout(3000)

        try:
            more_btns = page.locator("text=/Показать ещ[ёе]|Загрузить/i")
            if await more_btns.count() > 0:
                if await more_btns.first.is_visible():
                    await more_btns.first.click()
                    await page.wait_for_timeout(2000)
        except:
            pass

        if i % 2 == 0:
            print(f"   ...прокрутка {i+1}/10")

    content = await page.content()
    soup = BeautifulSoup(content, 'html.parser')

    events_list = []
    unique_keys = set()

    items = soup.find_all('div', class_='list-item')
    print(f"🔍 Найдено блоков .list-item: {len(items)}")

    for item in items:
        try:
            # 1. ЗАГОЛОВОК
            title_tag = item.find('div', class_='list-t')
            if not title_tag:
                continue
            title = title_tag.get_text(strip=True)

            # 2. ДАТА (Раздельные классы)
            date_raw = "Unknown"
            day_node = item.find('div', class_='list-day')
            month_node = item.find('div', class_='list-month')
            time_node = item.find('div', class_='list-time')

            if day_node and month_node:
                d_txt = day_node.get_text(strip=True)
                m_txt = month_node.get_text(strip=True)
                t_txt = time_node.get_text(strip=True) if time_node else ""
                date_raw = f"{d_txt} {m_txt} {t_txt}".strip()

            # 3. ССЫЛКА И СТАТУС
            url = ""
            status = "available"

            other_block = item.find('div', class_='list-other')
            if other_block:
                btn = other_block.find('a', href=True)
                if btn:
                    link = btn['href']
                    url = link if link.startswith('http') else f"{BASE_URL_SOBOR}{link}"
                    btn_text = btn.get_text(strip=True).upper()
                    if "SOLD" in btn_text or "ПРОДАНО" in btn_text:
                        status = "sold_out"

            if not url:
                status = "unknown"

            # 4. КАРТИНКА
            photo_url = ""
            img_block = item.find('div', class_='list-img')
            if img_block:
                img = img_block.find('img')
                if img and img.get('src'):
                    src = img['src']
                    if 'svg' not in src:
                        photo_url = src if src.startswith('http') else f"{BASE_URL_SOBOR}{src}"

            # 5. ОПИСАНИЕ (Markdown)
            description = ""
            descr_node = item.find('div', class_='list-descr')
            if descr_node:
                # get_text(separator="\n") заменяет <br> и закрывающие теги на перенос строки
                description = descr_node.get_text(separator="\n", strip=True)

            # Дедубликация
            key = f"{date_raw}_{title}"
            if key in unique_keys:
                continue
            unique_keys.add(key)

            events_list.append({
                "title": title,
                "date_raw": date_raw,
                "ticket_status": status,
                "url": url,
                "photos": [photo_url] if photo_url else [],
                "description": description,
                "pushkin_card": False,
                "location": "Кафедральный собор"
            })
        except Exception as e:
            print(f"⚠️ Ошибка при разборе элемента: {e}")
            continue

    print(f"✅ Успешно обработано событий: {len(events_list)}")
    return events_list

async def run_sobor(browser):
    context = await browser.new_context(viewport={'width': 1920, 'height': 1080})
    page = await context.new_page()

    data = await scrape_sobor_strict(page)
    await context.close()
    return data







# ==========================================
# ЧАСТЬ 3: ТРЕТЬЯКОВСКАЯ ГАЛЕРЕЯ
# ==========================================

BASE_URL_TRETYAKOV = "https://kaliningrad.tretyakovgallery.ru"

def normalize_tretyakov_date(date_raw):
    """
    Парсит дату из строк вида:
    - "Уже идет 4 января в 17:00" -> ("2025-01-04", "17:00")
    - "Скоро С 25 декабря по 10 января" -> ("2024-12-25", "00:00") (берем дату начала)
    """
    import datetime
    
    if not date_raw:
        return None, None

    text = date_raw.strip().lower()
    
    # 1. Извлекаем время
    time_match = re.search(r'(\d{1,2}):(\d{2})', text)
    parsed_time = "00:00"
    if time_match:
        parsed_time = f"{int(time_match.group(1)):02d}:{int(time_match.group(2)):02d}"

    # 2. Очистка от мусора
    text = re.sub(r"уже идет|скоро", "", text).strip()
    
    # 3. Поиск диапазона или перечисления
    range_match = re.search(r"с\s+(\d{1,2})\s+([а-я]+)", text)
    
    day = None
    month_name = None
    
    if range_match:
        day = int(range_match.group(1))
        month_name = range_match.group(2)
    else:
        # "2 и 8 января" - ищем любой месяц
        ru_months = ["января", "февраля", "марта", "апреля", "мая", "июня", 
                     "июля", "августа", "сентября", "октября", "ноября", "декабря"]
        
        found_month = None
        for m in ru_months:
            if m in text:
                found_month = m
                break
        
        if found_month:
            month_name = found_month
            # Ищем первое число
            digits = re.findall(r"\d+", text)
            if digits:
                day = int(digits[0])

    if not day or not month_name:
        simple = re.search(r"(\d{1,2})\s+([а-я]+)", text)
        if simple:
            day = int(simple.group(1))
            month_name = simple.group(2)

    # 4. Маппинг месяца
    MONTHS = {
        "января": 1, "февраля": 2, "марта", "апреля": 4, "мая": 5, "июня": 6,
        "июля": 7, "августа": 8, "сентября": 9, "октября": 10, "ноября": 11, "декабря": 12
    }
    month = MONTHS.get(month_name)
            
    if not month:
        return None, parsed_time

    # 5. Определение года
    today = datetime.date.today()
    year = today.year
    
    try:
        if today.month == 12 and month == 1:
             year += 1
        elif month < today.month and (today.month - month) > 4:
             year += 1
             
        event_date = datetime.date(year, month, day)
        return event_date.isoformat(), parsed_time
    except ValueError:
        return None, parsed_time


async def scrape_tretyakov_schedule(page):
    url = f"{BASE_URL_TRETYAKOV}/events/"
    print(f"\n🖼️ [Третьяковка] Этап 1: Сканируем события на {url}...")

    try:
        await page.goto(url, timeout=90000, wait_until='domcontentloaded')
        await page.wait_for_timeout(3000)

        for _ in range(3):
            await page.mouse.wheel(0, 4000)
            await page.wait_for_timeout(random.randint(1000, 2000))

        content = await page.content()
        soup = BeautifulSoup(content, 'html.parser')

        events_list = []
        seen_ids = set()

        cards = soup.select('.card')
        for card in cards:
            title_tag = card.select_one('.card_info .card_info-top .card_title')
            if not title_tag:
                continue
            title = title_tag.get_text(strip=True)

            link_tag = card.select_one('.card_info-top a[href]')
            href = link_tag.get('href') if link_tag else ""
            if href and href.startswith("//"):
                href = f"https:{href}"
            elif href and href.startswith('/'):
                href = f"{BASE_URL_TRETYAKOV}{href}"
            
            # --- ФОТО ---
            img_url = ""
            img_tag = card.select_one('img.card_img')
            if img_tag and img_tag.get('src'):
                img_url = img_tag['src']
            if not img_url:
                img_div = card.select_one('div.card_img')
                style = img_div.get('style', '') if img_div else ""
                match = re.search(r"url\((.*?)\)", style)
                if match:
                    img_url = match.group(1).strip().strip('"').strip("'")
            if img_url and img_url.startswith('/'):
                img_url = f"{BASE_URL_TRETYAKOV}{img_url}"

            # --- ДАТА ---
            date_text = ""
            info = card.select_one('.card_info')
            if info:
                bottom = info.select_one('.card_info-bottom') or info
                date_text = bottom.get_text(" ", strip=True)

            if title:
                date_text = date_text.replace(title, "").strip()

            clean_date = re.sub(r"Купить\s*билет|Купить|Билеты|Подробнее", "", date_text, flags=re.IGNORECASE).strip()
            clean_date = re.sub(r"\b(Атриум|Кинозал)\b", "", clean_date, flags=re.IGNORECASE).strip()
            
            parsed_date, parsed_time = normalize_tretyakov_date(clean_date)

            card_text = card.get_text(" ", strip=True).upper()
            if "АТРИУМ" in card_text:
                location = "Атриум"
            elif "КИНОЗАЛ" in card_text:
                location = "Кинозал"
            else:
                location = "Филиал Третьяковской галереи, Парадная наб. 3, #Калининград"

            event_id = f"{clean_date}_{title}_{href}"
            if event_id in seen_ids:
                continue
            seen_ids.add(event_id)

            events_list.append({
                "title": title,
                "date_raw": clean_date,
                "parsed_date": parsed_date,
                "parsed_time": parsed_time,
                "ticket_status": "unknown",
                "url": href,
                "photos": [img_url] if img_url else [],
                "location": location,
            })

        print(f"✅ [Третьяковка] Найдено событий: {len(events_list)}")
        return events_list
    except Exception as e:
        print(f"❌ [Третьяковка] Ошибка расписания: {e}")
        return []

async def _tretyakov_interact_and_get_price(page, target_date=None, target_time=None):
    """
    СТРАТЕГИЯ:
    1. Check Price (если есть - выход).
    2. Try STANDARD Selectors (exact classes) -> для большинства ивентов.
    3. Try FUZZY Selectors (JS lookup) -> для 'Fairy Tale' и сложных.
    4. Click Time -> Parse Price.
    """
    try:
        # 1. Wait for widget container or price
        # Широкий вейт, чтобы не падать сразу
        try:
            await page.wait_for_selector('div[class*="calendar"], .skin-inner, app-root, .wrapper, .page-buy-container', timeout=8000)
        except:
             pass

        # === CHECK 0: Pre-existing price ===
        text_init = await page.inner_text("body")
        if re.search(r"\d+\s*(?:₽|руб)", text_init):
            # Price visible?
            pass 

        await page.wait_for_timeout(1000)

        click_success = False
        target_day = None
        if target_date:
            try:
                target_day = int(target_date.split('-')[2])
            except:
                pass

        if target_day:
            print(f"   ...Target Day: {target_day}")
            
            # --- ATTEMPT 1: STANDARD SELECTORS (Strict) ---
            # Это работало для большинства событий ранее
            # Ищем элементы с классом day/cell и точным текстом
            try:
                # Selectors used in typical calendars
                candidates = await page.locator('.cell:not(.day-header), .day, .date-item, span[class*="day"]').all()
                for cand in candidates:
                    if not await cand.is_visible(): continue
                    
                    # Check text Exact Match
                    txt = (await cand.inner_text()).strip()
                    if txt == str(target_day):
                        # Found it!
                        cls = await cand.get_attribute('class') or ""
                        if "disabled" in cls or "sold" in cls: continue
                        
                        await cand.click(force=True)
                        click_success = True
                        print(f"   >>> [Standard] Clicked date {target_day}")
                        break
            except Exception as e:
                print(f"   Standard click error: {e}")

            # --- ATTEMPT 2: FUZZY SELECTORS (JS) ---
            # Если стандарт не сработал (как в Fairy Tale)
            if not click_success:
                print(f"   ...Attempts Fuzzy Search for {target_day}")
                try:
                    candidates = await page.evaluate_handle(r"""
                        (day) => {
                            const els = Array.from(document.querySelectorAll('div, span, button, td, a'));
                            const matches = els.filter(el => {
                                if (el.offsetParent === null) return false;
                                const cls = (el.className || "").toString();
                                if (cls.includes('disabled') || cls.includes('sold')) return false;
                                
                                const txt = el.innerText.trim();
                                // Regex: word boundary check for day number
                                // e.g. matches "29" in "ПН 29" but NOT "290" or "129"
                                const dayRx = new RegExp("(^|\\D)" + day + "(\\D|$)", "i");
                                if (!dayRx.test(txt)) return false;
                                
                                // Avoid long text blocks (descriptions)
                                if (txt.length > 50) return false;
                                
                                // Avoid obvious non-date things if possible?
                                return true;
                            });
                            // Prefer shorter matches (exact button vs container)
                            matches.sort((a,b) => a.innerText.length - b.innerText.length);
                            return matches.length > 0 ? matches[0] : null;
                        }
                    """, target_day)
                    
                    if candidates.as_element():
                        await candidates.as_element().click()
                        click_success = True
                        print(f"   >>> [Fuzzy] Clicked date {target_day}")
                except Exception as e:
                    print(f"   Fuzzy click error: {e}")

        # Fallback date (Any date 1-31) if target specific failed
        if not click_success:
             print("   ...Fallback: Clicking first available date 1-31")
             try:
                 candidates = await page.evaluate_handle(r"""
                   () => {
                        const els = Array.from(document.querySelectorAll('div, span, button, td, a'));
                        const matches = els.filter(el => {
                            if (el.offsetParent === null) return false;
                            const cls = (el.className || "").toString();
                            if (cls.includes('disabled') || cls.includes('sold')) return false;
                            
                            const txt = el.innerText.trim();
                            // STRICT: Matches exactly a number 1-31 alone, or matches "Mon 29" format
                            // Checking for explicit 1-31 range match
                            const dateMatch = txt.match(/(^|\D)([1-9]|[12]\d|3[01])(\D|$)/);
                            if (!dateMatch) return false;
                            
                            if (txt.length > 40) return false; 
                            return true;
                        });
                        matches.sort((a,b) => a.innerText.length - b.innerText.length);
                        return matches.length > 0 ? matches[0] : null;
                   }
                 """)
                 if candidates.as_element():
                     txt_click = await candidates.as_element().inner_text()
                     await candidates.as_element().click()
                     print(f"   >>> [Fallback] Clicked date: {txt_click[:10]}...")
             except Exception as e:
                 print(f"   Fallback error: {e}")

        # 3. TIME SELECTION
        await page.wait_for_timeout(1500) 
        
        target_hm = target_time if target_time else None
        print("   ...Searching Time")
        
        # Standard + Fuzzy Time Search
        try:
             time_el = await page.evaluate_handle(r"""
                (tgt) => {
                    const els = Array.from(document.querySelectorAll('div, span, button, td, a'));
                    const matches = els.filter(el => {
                        if (el.offsetParent === null) return false;
                        const cls = (el.className || "").toString();
                        if (cls.includes('disabled')) return false;
                        
                        const txt = el.innerText.trim();
                        // STRICT HH:MM pattern
                        const timeMatch = txt.match(/\d{1,2}:\d{2}/);
                        if (!timeMatch) return false;
                        
                        // If target provided, must contain it
                        if (tgt && !txt.includes(tgt)) return false;
                        
                        if (txt.length > 30) return false;
                        return true;
                    });
                     matches.sort((a,b) => a.innerText.length - b.innerText.length);
                     return matches.length > 0 ? matches[0] : null;
                }
             """, target_hm)
             
             if time_el.as_element():
                 t_txt = await time_el.as_element().inner_text()
                 await time_el.as_element().click()
                 print(f"   >>> Clicked time: {t_txt}")
        except:
            pass

        # 4. PRICE EXTRACTION
        await page.wait_for_timeout(2000)

        text = await page.inner_text("body")
        lower = text.lower()
        
        if "нет билетов" in lower or "sold out" in lower or "распродано" in lower:
            return "sold_out", None, None
            
        prices = []
        matches = re.findall(r"(\d+)\s*(?:₽|руб|р\.)", text, flags=re.IGNORECASE)
        for m in matches:
            prices.append(int(m))
            
        if prices:
            prices = sorted(set(prices))
            return "available", prices[0], prices[-1]
            
        return "unknown", None, None

    except Exception as e:
        print(f"   ⚠️ Interaction error: {e}")
        return "unknown", None, None


async def scrape_tretyakov_ticket_price(context, url, parsed_date=None, parsed_time=None):
    print(f"💳 [Третьяковка] Проверяем: {url}")
    if not url: return {"ticket_status": "unknown", "ticket_price_min": None, "ticket_price_max": None}

    page = await context.new_page()
    status, p_min, p_max = "unknown", None, None

    try:
        await page.goto(url, timeout=60000, wait_until='domcontentloaded')
        status, p_min, p_max = await _tretyakov_interact_and_get_price(page, parsed_date, parsed_time)
        print(f"   Result: {status} Price: {p_min}-{p_max}")
    except Exception as e:
        print(f"❌ Error: {e}")
    finally:
        await page.close()
    
    return {
        "ticket_status": status,
        "ticket_price_min": p_min,
        "ticket_price_max": p_max,
    }

async def run_tretyakov(browser):
    context = await browser.new_context(viewport={'width': 1920, 'height': 1080})
    await context.route("**/*{google,yandex,metrika,analytics}*", lambda route: route.abort())
    page = await context.new_page()

    schedule = await scrape_tretyakov_schedule(page)
    if not schedule:
        await context.close()
        return []

    for item in schedule:
        url = item.get("url", "")
        if not url or "tickets" not in url: continue
        
        res = await scrape_tretyakov_ticket_price(context, url, item.get("parsed_date"), item.get("parsed_time"))
        item.update(res)
        await page.wait_for_timeout(random.randint(500, 1000))

    await context.close()
    return schedule


# --- ЗАПУСК ---
async def main():
    config = {}
    target = "all"
    if os.path.exists('run_config.json'):
        config = json.load(open('run_config.json'))
        target = config.get('target_source', 'all')

    print("🚀 ЗАПУСК СБОРА ДАННЫХ...")
    print("ℹ️ [INFO] Запускаем Драмтеатр, Музтеатр, Кафедральный собор и Третьяковскую галерею.")

    results = {}

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)

        data_dram = []
        data_muz = []
        data_sobor = []
        data_tretyakov = []

        if target in ("all", "dramteatr"):
            data_dram = await run_dramteatr(browser)
            results["dramteatr"] = data_dram
            if data_dram:
                with open('dramteatr.json', 'w', encoding='utf-8') as f:
                    json.dump(data_dram, f, ensure_ascii=False, indent=4)
                pd.DataFrame(data_dram).to_csv('dramteatr.csv', index=False)
                print(f"💾 Dramteatr saved: {len(data_dram)}")

        if target in ("all", "muzteatr"):
            data_muz = await run_muzteatr(browser)
            results["muzteatr"] = data_muz
            if data_muz:
                with open('muzteatr.json', 'w', encoding='utf-8') as f:
                    json.dump(data_muz, f, ensure_ascii=False, indent=4)
                df_muz = pd.DataFrame(data_muz)
                if 'photos' in df_muz.columns:
                    df_muz['photos'] = df_muz['photos'].apply(lambda x: ' | '.join(x) if isinstance(x, list) else x)
                df_muz.to_csv('muzteatr.csv', index=False)
                print(f"💾 Muzteatr saved: {len(data_muz)}")

        if target in ("all", "sobor"):
            data_sobor = await run_sobor(browser)
            results["sobor"] = data_sobor

        if target in ("all", "tretyakov"):
            data_tretyakov = await run_tretyakov(browser)
            results["tretyakov"] = data_tretyakov

        await browser.close()

    all_data = []
    for chunk in results.values():
        if chunk:
            all_data.extend(chunk)

    if all_data:
        results["all"] = all_data

    return results

result = await main()

sobor_data = result.get("sobor", []) if result else []
tretyakov_data = result.get("tretyakov", []) if result else []
all_data = result.get("all", []) if result else []

if sobor_data:
    with open('sobor.json', 'w', encoding='utf-8') as f:
        json.dump(sobor_data, f, ensure_ascii=False, indent=4)

    df = pd.DataFrame(sobor_data)
    if 'photos' in df.columns:
        df['photo_preview'] = df['photos'].apply(lambda x: x[0] if x else "")

    # Обрезаем описание для превью в консоли
    if 'description' in df.columns:
        df['desc_preview'] = df['description'].apply(lambda x: (x[:50] + '...') if len(x) > 50 else x)

    df.drop(columns=['photos', 'desc_preview'], inplace=True, errors='ignore')
    df.to_csv('sobor.csv', index=False)

if tretyakov_data:
    with open('tretyakov.json', 'w', encoding='utf-8') as f:
        json.dump(tretyakov_data, f, ensure_ascii=False, indent=4)

    df_tretyakov = pd.DataFrame(tretyakov_data)
    if 'photos' in df_tretyakov.columns:
        df_tretyakov['photos'] = df_tretyakov['photos'].apply(lambda x: ' | '.join(x) if isinstance(x, list) else x)
    df_tretyakov.to_csv('tretyakov.csv', index=False)

if all_data:
    df_all = pd.DataFrame(all_data)
    if 'photos' in df_all.columns:
        df_all['photo_preview'] = df_all['photos'].apply(lambda x: x[0] if isinstance(x, list) and x else "")
    if 'description' in df_all.columns:
        df_all['desc_preview'] = df_all['description'].apply(lambda x: (x[:50] + '...') if len(x) > 50 else x)
    else:
        df_all['desc_preview'] = ""

    print(f"\n🎉 ИТОГ: {len(all_data)} событий.")
    pd.set_option('display.max_colwidth', 40)
    print(df_all[['date_raw', 'title', 'desc_preview']].head(10))
else:
    print("Данные не найдены.")
